The goal of this notebook is to change the tables of data into readable ones so as to easily create visualizations in PowerBI.

In [57]:
#Dependencies

import pandas as pd


In [227]:
#Load all data

sleep_df = pd.read_csv('../data/landing/HKCategoryTypeIdentifierSleepAnalysis.csv', sep=";", header=1)
active_energy_df = pd.read_csv('../data/landing/HKQuantityTypeIdentifierActiveEnergyBurned.csv', sep=";", header=1)
stand_df = pd.read_csv('../data/landing/HKQuantityTypeIdentifierAppleStandTime.csv', sep=";", header=1)
basal_energy_df = pd.read_csv('../data/landing/HKQuantityTypeIdentifierBasalEnergyBurned.csv', sep=";", header=1)
step_df = pd.read_csv('../data/landing/HKQuantityTypeIdentifierStepCount.csv', sep=";", header=1)
walk_speed_df = pd.read_csv('../data/landing/HKQuantityTypeIdentifierWalkingSpeed.csv', sep=";", header=1)

In [165]:
#Start with the sleep dataset

sleep_df.head()

,type,sourcename,sourceversion,creationdate,startdate,enddate,value,HKTimeZone,HKWasUserEntered
0,HKCategoryTypeIdentifierSleepAnalysis,Matthew’s iPhone Pro Max,14.4.2,2021-04-11 11:00:23 +1100,2021-04-11 04:27:46 +1100,2021-04-11 11:00:23 +1100,HKCategoryValueSleepAnalysisInBed,Asia/Manila,NaN
1,HKCategoryTypeIdentifierSleepAnalysis,Matthew’s iPhone Pro Max,14.4.2,2021-04-12 11:09:48 +1100,2021-04-12 03:00:00 +1100,2021-04-12 11:05:27 +1100,HKCategoryValueSleepAnalysisInBed,Asia/Manila,NaN
2,HKCategoryTypeIdentifierSleepAnalysis,Matthew’s iPhone Pro Max,14.4.2,2021-04-13 11:09:38 +1100,2021-04-13 03:06:22 +1100,2021-04-13 11:09:38 +1100,HKCategoryValueSleepAnalysisInBed,Asia/Manila,NaN
3,HKCategoryTypeIdentifierSleepAnalysis,Matthew’s iPhone Pro Max,14.4.2,2021-04-14 11:00:11 +1100,2021-04-14 03:00:00 +1100,2021-04-14 11:00:11 +1100,HKCategoryValueSleepAnalysisInBed,Asia/Manila,NaN
4,HKCategoryTypeIdentifierSleepAnalysis,Matthew’s iPhone Pro Max,14.4.2,2021-04-15 11:00:12 +1100,2021-04-15 03:36:51 +1100,2021-04-15 11:00:12 +1100,HKCategoryValueSleepAnalysisInBed,Asia/Manila,NaN


In [166]:
sleep_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35476 entries, 0 to 35475
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   type              35476 non-null  object 
 1   sourcename        35476 non-null  object 
 2   sourceversion     35476 non-null  object 
 3   creationdate      35476 non-null  object 
 4   startdate         35476 non-null  object 
 5   enddate           35476 non-null  object 
 6   value             35476 non-null  object 
 7   HKTimeZone        31741 non-null  object 
 8   HKWasUserEntered  4 non-null      float64
dtypes: float64(1), object(8)
memory usage: 2.4+ MB


In [242]:
#Since the data will always come in the same format, we can create a function to wrangle it once and for all.

def wrangle_sleep(df):
    

    #Remove unnecessary columns and renaming.
    df = df.drop(columns=['type', 'sourceversion', 'HKWasUserEntered']).rename(columns={'creationdate':'date'})

    #Remove values from Mi Fit and phone, as we only want data from the apple watch. Plus, both track the same thing and we don't want to pollute the data with variance.
    df = df[df['sourcename'] == 'Matthew Lance’s Apple\xa0Watch']

    #Improve readability
    df['value'] = df['value'].str.replace('HKCategoryValueSleepAnalysis', '', regex=False)
    df['sourcename'] = df['sourcename'].str.replace('Matthew Lance’s Apple\xa0Watch', 'AppleWatch', regex=False)

    #Transform date columns type to datetime, and we only need the date from 'date'.
    date_cols = ['date', 'startdate', 'enddate']
    df[date_cols] = df[date_cols].apply(pd.to_datetime)

    #Create a new column 'totaldurationhours', which shows the duration of time in hours between startdate and enddate.
    df.insert(5, 'totaldurationhours', None)

    #Fill 'totaldurationhours' with values.
    df['totaldurationhours'] = (df['enddate'] - df['startdate']).dt.total_seconds() / 3600

    return df

In [243]:
sleep_wrangled = wrangle_sleep(sleep_df)
sleep_wrangled.head()

,sourcename,date,startdate,enddate,value,totaldurationhours,HKTimeZone
3370,AppleWatch,2021-08-12 11:08:16+11:00,2021-08-12 03:16:51+11:00,2021-08-12 06:04:21+11:00,AsleepUnspecified,2.791667,NaN
3371,AppleWatch,2021-08-12 11:08:16+11:00,2021-08-12 06:05:21+11:00,2021-08-12 07:43:51+11:00,AsleepUnspecified,1.641667,NaN
3372,AppleWatch,2021-08-12 11:08:16+11:00,2021-08-12 07:45:21+11:00,2021-08-12 09:13:51+11:00,AsleepUnspecified,1.475000,NaN
3373,AppleWatch,2021-08-12 11:08:16+11:00,2021-08-12 09:16:21+11:00,2021-08-12 09:17:51+11:00,AsleepUnspecified,0.025000,NaN
3374,AppleWatch,2021-08-12 11:08:16+11:00,2021-08-12 09:18:51+11:00,2021-08-12 09:36:21+11:00,AsleepUnspecified,0.291667,NaN


In [244]:
#Create a second function where I can really track everything without worry of losing data. 

def extract_sleep(df):
    
    #Simplify 'date' and remove 'startdate' and 'enddate'. 
    df['date'] = df['date'].dt.date
    df = df.drop(columns=['startdate', 'enddate'])

    #Groupby 'value' and aggregate 'totaldurationhours'
    df = df.groupby(['date', 'value'])['totaldurationhours'].sum().reset_index().round(2)

    #Create another df for sleep per day.
    sleep_day = df.groupby('date')['totaldurationhours'].sum().reset_index().round(2)
    sleep_day['day_of_week'] = pd.to_datetime(sleep_day['date']).dt.day_name()

    return df, sleep_day

In [245]:
#This is precisely what I was trying to extract.

sleep_value, sleep_per_day = extract_sleep(sleep_wrangled)
print(sleep_value)
print(sleep_per_day)

            date              value  totaldurationhours
0     2021-08-12  AsleepUnspecified                7.42
1     2021-08-13  AsleepUnspecified                6.67
2     2021-08-14  AsleepUnspecified                7.36
3     2021-08-15  AsleepUnspecified                6.47
4     2021-08-16  AsleepUnspecified                7.40
...          ...                ...                 ...
2974  2024-10-19              Awake                0.07
2975  2024-10-20         AsleepCore                4.54
2976  2024-10-20         AsleepDeep                0.84
2977  2024-10-20          AsleepREM                1.48
2978  2024-10-20              Awake                0.10

[2979 rows x 3 columns]
           date  totaldurationhours day_of_week
0    2021-08-12                7.42    Thursday
1    2021-08-13                6.67      Friday
2    2021-08-14                7.36    Saturday
3    2021-08-15                6.47      Sunday
4    2021-08-16                7.40      Monday
..          ...

In [246]:
sleep_value.to_csv('../data/raw/matthew_sleep_type.csv', sep=',', index=False)
sleep_per_day.to_csv('../data/raw/matthew_sleep_per_day.csv', sep=',', index=False)

In [178]:
#Now for the energy datasets, same concept as with the sleep dataset.

active_energy_df.head()

,type,sourcename,sourceversion,device,unit,creationdate,startdate,enddate,value
0,HKQuantityTypeIdentifierActiveEnergyBurned,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd180>, name:Apple Watch, ma...",Cal,2021-11-08 12:14:50 +1100,2021-11-08 12:03:49 +1100,2021-11-08 12:13:32 +1100,1.104
1,HKQuantityTypeIdentifierActiveEnergyBurned,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd180>, name:Apple Watch, ma...",Cal,2021-11-08 12:26:07 +1100,2021-11-08 12:14:44 +1100,2021-11-08 12:24:37 +1100,2.360
2,HKQuantityTypeIdentifierActiveEnergyBurned,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd180>, name:Apple Watch, ma...",Cal,2021-11-08 12:37:45 +1100,2021-11-08 12:24:58 +1100,2021-11-08 12:34:51 +1100,4.492
3,HKQuantityTypeIdentifierActiveEnergyBurned,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd180>, name:Apple Watch, ma...",Cal,2021-11-08 12:47:19 +1100,2021-11-08 12:34:51 +1100,2021-11-08 12:44:46 +1100,8.193
4,HKQuantityTypeIdentifierActiveEnergyBurned,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd180>, name:Apple Watch, ma...",Cal,2021-11-08 12:57:55 +1100,2021-11-08 12:44:46 +1100,2021-11-08 12:54:19 +1100,3.329


In [179]:
active_energy_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 194625 entries, 0 to 194624
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype  
---  ------         --------------   -----  
 0   type           194625 non-null  object 
 1   sourcename     194625 non-null  object 
 2   sourceversion  194625 non-null  object 
 3   device         187395 non-null  object 
 4   unit           194625 non-null  object 
 5   creationdate   194625 non-null  object 
 6   startdate      194625 non-null  object 
 7   enddate        194625 non-null  object 
 8   value          194625 non-null  float64
dtypes: float64(1), object(8)
memory usage: 13.4+ MB


In [228]:
def wrangle_energy(df):

    #Retrieve only data from the Apple Watch.
    df = df[df['sourcename'] == 'Matthew Lance’s Apple\xa0Watch']

    #Remove unnecessary columns
    df = df.drop(columns=['type', 'sourcename', 'sourceversion', 'device', 'unit']).rename(columns={'creationdate':'date'})

    #Convert date columns into datetime.
    date_cols = ['date', 'startdate', 'enddate']
    df[date_cols] = df[date_cols].apply(pd.to_datetime)

    return df

In [186]:
active_wrangled = wrangle_energy(active_energy_df)
active_wrangled

,date,startdate,enddate,value
0,2021-11-08 12:14:50+11:00,2021-11-08 12:03:49+11:00,2021-11-08 12:13:32+11:00,1.104
1,2021-11-08 12:26:07+11:00,2021-11-08 12:14:44+11:00,2021-11-08 12:24:37+11:00,2.360
2,2021-11-08 12:37:45+11:00,2021-11-08 12:24:58+11:00,2021-11-08 12:34:51+11:00,4.492
3,2021-11-08 12:47:19+11:00,2021-11-08 12:34:51+11:00,2021-11-08 12:44:46+11:00,8.193
4,2021-11-08 12:57:55+11:00,2021-11-08 12:44:46+11:00,2021-11-08 12:54:19+11:00,3.329
...,...,...,...,...
194620,2024-10-20 22:26:33+11:00,2024-10-20 22:25:14+11:00,2024-10-20 22:25:34+11:00,1.625
194621,2024-10-20 22:35:57+11:00,2024-10-20 22:25:34+11:00,2024-10-20 22:35:38+11:00,16.085
194622,2024-10-20 22:47:15+11:00,2024-10-20 22:35:38+11:00,2024-10-20 22:45:43+11:00,3.633
194623,2024-10-20 22:55:57+11:00,2024-10-20 22:45:43+11:00,2024-10-20 22:46:54+11:00,0.540


In [230]:
#Extract the important information from active_wrangled.

def extract_energy(df):

    #Extract only YYYY-MM-DD from 'date'.
    df['date'] = df['date'].dt.date
    df = df.drop(columns=['startdate', 'enddate'])

    #Aggregate on individual days.
    df = df.groupby('date')['value'].sum().reset_index().round(2)

    #Add new column 'day_of_week'
    df['day_of_week'] = pd.to_datetime(df['date']).dt.day_name()

    return df

In [231]:
active_energy_day = extract_energy(active_wrangled)
active_energy_day.to_csv('../data/raw/matthew_active_day.csv', sep=',', index=False)

In [232]:
active_energy_day

,date,value,day_of_week
0,2021-08-12,354.57,Thursday
1,2021-08-13,358.70,Friday
2,2021-08-14,506.83,Saturday
3,2021-08-15,380.34,Sunday
4,2021-08-16,358.82,Monday
...,...,...,...
1117,2024-10-16,844.93,Wednesday
1118,2024-10-17,847.95,Thursday
1119,2024-10-18,1023.76,Friday
1120,2024-10-19,954.56,Saturday


In [197]:
step_df

,type,sourcename,sourceversion,device,unit,creationdate,startdate,enddate,value
0,HKQuantityTypeIdentifierStepCount,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd040>, name:Apple Watch, ma...",count,2021-11-08 12:22:13 +1100,2021-11-08 12:11:04 +1100,2021-11-08 12:11:17 +1100,14
1,HKQuantityTypeIdentifierStepCount,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd040>, name:Apple Watch, ma...",count,2021-11-08 12:45:29 +1100,2021-11-08 12:32:50 +1100,2021-11-08 12:42:27 +1100,165
2,HKQuantityTypeIdentifierStepCount,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd040>, name:Apple Watch, ma...",count,2021-11-08 12:57:55 +1100,2021-11-08 12:46:03 +1100,2021-11-08 12:46:05 +1100,8
3,HKQuantityTypeIdentifierStepCount,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd040>, name:Apple Watch, ma...",count,2021-11-08 13:08:44 +1100,2021-11-08 12:58:04 +1100,2021-11-08 13:07:31 +1100,230
4,HKQuantityTypeIdentifierStepCount,Matthew Lance’s Apple Watch,8.3,"<<HKDevice: 0x3001cd040>, name:Apple Watch, ma...",count,2021-11-08 13:21:22 +1100,2021-11-08 13:07:34 +1100,2021-11-08 13:16:17 +1100,128
...,...,...,...,...,...,...,...,...,...
73836,HKQuantityTypeIdentifierStepCount,Matthew Lance’s Apple Watch,11.0.1,"<<HKDevice: 0x3001cd3b0>, name:Apple Watch, ma...",count,2024-10-20 22:05:46 +1100,2024-10-20 21:52:53 +1100,2024-10-20 22:01:02 +1100,360
73837,HKQuantityTypeIdentifierStepCount,Matthew Lance’s Apple Watch,11.0.1,"<<HKDevice: 0x3001cd3b0>, name:Apple Watch, ma...",count,2024-10-20 22:15:53 +1100,2024-10-20 22:05:46 +1100,2024-10-20 22:14:49 +1100,317
73838,HKQuantityTypeIdentifierStepCount,Matthew Lance’s Apple Watch,11.0.1,"<<HKDevice: 0x3001cd3b0>, name:Apple Watch, ma...",count,2024-10-20 22:26:33 +1100,2024-10-20 22:15:48 +1100,2024-10-20 22:25:47 +1100,967
73839,HKQuantityTypeIdentifierStepCount,Matthew Lance’s Apple Watch,11.0.1,"<<HKDevice: 0x3001cd3b0>, name:Apple Watch, ma...",count,2024-10-20 22:35:57 +1100,2024-10-20 22:25:47 +1100,2024-10-20 22:33:00 +1100,403


In [212]:
def wrangle_step(df):

    #Retrieve only data from Apple Watch.
    df = df[df['sourcename'] == 'Matthew Lance’s Apple\xa0Watch']

    #Remove unnecessary columns
    df = df.drop(columns=['type', 'sourceversion', 'sourcename', 'device', 'unit']).rename(columns={'creationdate':'date'})

    #Convert date columns into datetime.
    date_cols = ['date', 'startdate', 'enddate']
    df[date_cols] = df[date_cols].apply(pd.to_datetime)
    
    return df

In [213]:
step_wrangled = wrangle_step(step_df)
step_wrangled

,date,startdate,enddate,value
0,2021-11-08 12:22:13+11:00,2021-11-08 12:11:04+11:00,2021-11-08 12:11:17+11:00,14
1,2021-11-08 12:45:29+11:00,2021-11-08 12:32:50+11:00,2021-11-08 12:42:27+11:00,165
2,2021-11-08 12:57:55+11:00,2021-11-08 12:46:03+11:00,2021-11-08 12:46:05+11:00,8
3,2021-11-08 13:08:44+11:00,2021-11-08 12:58:04+11:00,2021-11-08 13:07:31+11:00,230
4,2021-11-08 13:21:22+11:00,2021-11-08 13:07:34+11:00,2021-11-08 13:16:17+11:00,128
...,...,...,...,...
73836,2024-10-20 22:05:46+11:00,2024-10-20 21:52:53+11:00,2024-10-20 22:01:02+11:00,360
73837,2024-10-20 22:15:53+11:00,2024-10-20 22:05:46+11:00,2024-10-20 22:14:49+11:00,317
73838,2024-10-20 22:26:33+11:00,2024-10-20 22:15:48+11:00,2024-10-20 22:25:47+11:00,967
73839,2024-10-20 22:35:57+11:00,2024-10-20 22:25:47+11:00,2024-10-20 22:33:00+11:00,403


In [214]:
def extract_step(df):

    #Extract only YYYY-MM-DD from 'date'.
    df['date'] = df['date'].dt.date
    df = df.drop(columns=['startdate', 'enddate'])

    #Aggregate on individual days.
    df = df.groupby('date')['value'].sum().reset_index().round(2)

    #Add new column 'day_of_week'
    df['day_of_week'] = pd.to_datetime(df['date']).dt.day_name()

    return df

In [215]:
step_per_day = extract_step(step_wrangled)
step_per_day.to_csv('../data/raw/matthew_step_day.csv', sep=',', index=False)

In [235]:
basal_wrangled = wrangle_energy(basal_energy_df)
basal_per_day = extract_energy(basal_wrangled)
basal_per_day['value'] = (basal_per_day['value'] / 4.184).round(2)
basal_per_day.to_csv('../data/raw/matthew_basal_day.csv', sep=',', index=False)

In [236]:
basal_per_day

,date,value,day_of_week
0,2021-08-12,1660.58,Thursday
1,2021-08-13,1813.35,Friday
2,2021-08-14,1813.72,Saturday
3,2021-08-15,1806.34,Sunday
4,2021-08-16,1796.63,Monday
...,...,...,...
1118,2024-10-16,1981.79,Wednesday
1119,2024-10-17,2011.07,Thursday
1120,2024-10-18,1903.02,Friday
1121,2024-10-19,1893.51,Saturday
